In [7]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration
BASE_PATH = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized"
OUTPUT_PATH = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/stretched"

# Filter names
FILTERS = ['clear', 'g', 'r', 'i']
OBJECT_NAME = "NGC1365"

# Stretching parameters (adjust these as needed)
STRETCH_PARAMS = {
    'log': {
        'factor': 1,  # Higher = more aggressive stretch
    },
    'hyperbolic': {
        'beta': 1.0,  # 0.1-10, lower = softer stretch
    },
    'asinh': {
        'softening': 1.0,  # 0.01-100, higher = softer transition
    }
}

# Percentile clipping
CLIP_PERCENTILE = (0.5, 99.5)


def load_normalized_image(filter_name):
    """Load normalized FITS image"""
    filename = f"normalized_{OBJECT_NAME}_{filter_name}_stacked_sigma3.0.fits"
    filepath = Path(BASE_PATH) / filename
    
    with fits.open(filepath) as hdul:
        data = hdul[0].data.astype(np.float64)
        header = hdul[0].header
    
    print(f"Loaded {filter_name}: shape={data.shape}, dtype={data.dtype}")
    print(f"  Range: [{np.nanmin(data):.6f}, {np.nanmax(data):.6f}]")
    
    return data, header


def clip_data(data, percentile=(0.5, 99.5)):
    """Clip data to percentile range"""
    vmin, vmax = np.nanpercentile(data, percentile)
    clipped = np.clip(data, vmin, vmax)
    
    # Normalize to 0-1 range
    clipped = (clipped - vmin) / (vmax - vmin)
    
    print(f"  Clipped to percentiles {percentile}: [{vmin:.6f}, {vmax:.6f}]")
    print(f"  After normalization: [{np.nanmin(clipped):.6f}, {np.nanmax(clipped):.6f}]")
    
    return clipped, vmin, vmax


def logarithmic_stretch(data, factor=1):
    """
    Logarithmic stretch - Good for faint details
    Formula: stretched = log(1 + factor * data) / log(1 + factor)
    """
    stretched = np.log(1 + factor * data) / np.log(1 + factor)
    return stretched


def hyperbolic_stretch(data, beta=1.0):
    """
    Hyperbolic stretch - Softer than log, preserves bright features
    Formula: stretched = data / sqrt(data^2 + beta^2)
    Normalized version for better visualization
    """
    stretched = data / np.sqrt(data**2 + beta**2)
    # Normalize to 0-1
    stretched = (stretched - np.nanmin(stretched)) / (np.nanmax(stretched) - np.nanmin(stretched))
    return stretched


def asinh_stretch(data, softening=1.0):
    """
    Asinh (arcsinh) stretch - Best for wide dynamic range
    Formula: stretched = arcsinh(data * softening) / arcsinh(softening)
    """
    stretched = np.arcsinh(data * softening) / np.arcsinh(softening)
    return stretched


def apply_all_stretches(data, params):
    """Apply all three stretch methods"""
    stretches = {}
    
    print("  Applying logarithmic stretch...")
    stretches['log'] = logarithmic_stretch(data, factor=params['log']['factor'])
    
    print("  Applying hyperbolic stretch...")
    stretches['hyperbolic'] = hyperbolic_stretch(data, beta=params['hyperbolic']['beta'])
    
    print("  Applying asinh stretch...")
    stretches['asinh'] = asinh_stretch(data, softening=params['asinh']['softening'])
    
    return stretches


def save_stretched_images(stretches, filter_name, header, output_path):
    """Save stretched images as FITS files"""
    Path(output_path).mkdir(parents=True, exist_ok=True)
    
    for method, stretched_data in stretches.items():
        filename = f"stretched_{method}_{OBJECT_NAME}_{filter_name}.fits"
        filepath = Path(output_path) / filename
        
        # Update header
        new_header = header.copy()
        new_header['STRETCH'] = method.upper()
        new_header['HISTORY'] = f'Applied {method} stretch'
        
        # Save
        fits.writeto(filepath, stretched_data.astype(np.float32), new_header, overwrite=True)
        print(f"  Saved: {filename}")


def create_comparison_plot(original, stretches, filter_name, output_path):
    """Create comparison visualization"""
    fig, axes = plt.subplots(2, 2, figsize=(16, 16))
    fig.suptitle(f'{OBJECT_NAME} - {filter_name.upper()} Filter Comparison', 
                 fontsize=16, fontweight='bold')
    
    # Original (clipped and normalized)
    im0 = axes[0, 0].imshow(original, cmap='gray', origin='lower', vmin=0, vmax=1)
    axes[0, 0].set_title('Original (Clipped & Normalized)', fontsize=14)
    axes[0, 0].axis('off')
    plt.colorbar(im0, ax=axes[0, 0], fraction=0.046)
    
    # Logarithmic
    im1 = axes[0, 1].imshow(stretches['log'], cmap='gray', origin='lower', vmin=0, vmax=1)
    axes[0, 1].set_title(f'Logarithmic (factor={STRETCH_PARAMS["log"]["factor"]})', fontsize=14)
    axes[0, 1].axis('off')
    plt.colorbar(im1, ax=axes[0, 1], fraction=0.046)
    
    # Hyperbolic
    im2 = axes[1, 0].imshow(stretches['hyperbolic'], cmap='gray', origin='lower', vmin=0, vmax=1)
    axes[1, 0].set_title(f'Hyperbolic (beta={STRETCH_PARAMS["hyperbolic"]["beta"]})', fontsize=14)
    axes[1, 0].axis('off')
    plt.colorbar(im2, ax=axes[1, 0], fraction=0.046)
    
    # Asinh
    im3 = axes[1, 1].imshow(stretches['asinh'], cmap='gray', origin='lower', vmin=0, vmax=1)
    axes[1, 1].set_title(f'Asinh (softening={STRETCH_PARAMS["asinh"]["softening"]})', fontsize=14)
    axes[1, 1].axis('off')
    plt.colorbar(im3, ax=axes[1, 1], fraction=0.046)
    
    plt.tight_layout()
    
    # Save comparison
    comparison_file = Path(output_path) / f"comparison_{OBJECT_NAME}_{filter_name}.png"
    plt.savefig(comparison_file, dpi=150, bbox_inches='tight')
    print(f"  Saved comparison: {comparison_file.name}")
    plt.close()


def process_filter(filter_name):
    """Process one filter through all stretches"""
    print(f"\n{'='*60}")
    print(f"Processing {filter_name.upper()} filter")
    print(f"{'='*60}")
    
    # Load image
    data, header = load_normalized_image(filter_name)
    
    # Clip and normalize
    clipped_data, vmin, vmax = clip_data(data, percentile=CLIP_PERCENTILE)
    
    # Apply stretches
    stretches = apply_all_stretches(clipped_data, STRETCH_PARAMS)
    
    # Save stretched images
    print("\nSaving stretched FITS files...")
    save_stretched_images(stretches, filter_name, header, OUTPUT_PATH)
    
    # Create comparison plot
    print("\nCreating comparison plot...")
    create_comparison_plot(clipped_data, stretches, filter_name, OUTPUT_PATH)
    
    print(f"\n✓ Completed {filter_name} filter")
    
    return stretches


def main():
    """Main processing function"""
    print("="*60)
    print("GALAXY IMAGE STRETCHING - NGC1365")
    print("="*60)
    print(f"\nInput path: {BASE_PATH}")
    print(f"Output path: {OUTPUT_PATH}")
    print(f"\nStretch parameters:")
    print(f"  Logarithmic factor: {STRETCH_PARAMS['log']['factor']}")
    print(f"  Hyperbolic beta: {STRETCH_PARAMS['hyperbolic']['beta']}")
    print(f"  Asinh softening: {STRETCH_PARAMS['asinh']['softening']}")
    print(f"\nPercentile clipping: {CLIP_PERCENTILE}")
    
    # Process all filters
    all_stretches = {}
    for filter_name in FILTERS:
        try:
            stretches = process_filter(filter_name)
            all_stretches[filter_name] = stretches
        except Exception as e:
            print(f"\n✗ Error processing {filter_name}: {str(e)}")
            continue
    
    print("\n" + "="*60)
    print("PROCESSING COMPLETE!")
    print("="*60)
    print(f"\nOutput files saved to: {OUTPUT_PATH}")
    print("\nNext steps:")
    print("1. Review comparison PNG files to choose best stretch method")
    print("2. Use chosen stretched FITS files for alignment")
    print("3. Create color composites after alignment")
    print("\nTo adjust parameters, modify STRETCH_PARAMS at the top of the code")


if __name__ == "__main__": 
    main()

GALAXY IMAGE STRETCHING - NGC1365

Input path: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized
Output path: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/stretched

Stretch parameters:
  Logarithmic factor: 1
  Hyperbolic beta: 1.0
  Asinh softening: 1.0

Percentile clipping: (0.5, 99.5)

Processing CLEAR filter
Loaded clear: shape=(2048, 2048), dtype=float64
  Range: [0.000000, 1.000000]
  Clipped to percentiles (0.5, 99.5): [0.018566, 0.022598]
  After normalization: [0.000000, 1.000000]
  Applying logarithmic stretch...
  Applying hyperbolic stretch...
  Applying asinh stretch...

Saving stretched FITS files...
  Saved: stretched_log_NGC1365_clear.fits
  Saved: stretched_hyperbolic_NGC1365_clear.fits
  Saved: stretched_asinh_NGC1365_clear.fits

Creating comparison plot...
  Saved comparison: comparison_NGC1365_clear.png

✓ Completed clear filter

Processing 

In [28]:
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Configuration
BASE_PATH = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized"
OUTPUT_PATH = "/home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/stretched"

# Filter names
FILTERS = ['clear', 'g', 'r', 'i']
OBJECT_NAME = "NGC1365"

# FIXED stretching parameters
STRETCH_PARAMS = {
    'log': {
        'factor': 100,
    },
    'hyperbolic': {
        'beta': 0.5,
    },
    'asinh': {
        'softening': 0.1,
    },
    'gamma': {
        'gamma': 2.2,
    },
    'adaptive_asinh': {
        'q': 0.01,        # FIXED: was 0.01, now more aggressive
        'stretch': 1,   
    }
}

# Less aggressive clipping to preserve dynamic range
CLIP_PERCENTILE = (0.1, 99.9)


def load_normalized_image(filter_name):
    """Load normalized FITS image"""
    filename = f"normalized_{OBJECT_NAME}_{filter_name}_stacked_sigma3.0.fits"
    filepath = Path(BASE_PATH) / filename
    
    with fits.open(filepath) as hdul:
        data = hdul[0].data.astype(np.float64)
        header = hdul[0].header
    
    print(f"Loaded {filter_name}: shape={data.shape}, dtype={data.dtype}")
    print(f"  Range: [{np.nanmin(data):.6f}, {np.nanmax(data):.6f}]")
    print(f"  Median: {np.nanmedian(data):.6f}, Std: {np.nanstd(data):.6f}")
    
    return data, header


def normalize_data(data, percentile=(0.1, 99.9)):
    """Normalize data with percentile clipping"""
    # Replace NaN with median
    data = np.nan_to_num(data, nan=np.nanmedian(data))
    
    # Subtract background (use low percentile)
    background = np.percentile(data, 0.5)
    data_sub = data - background
    data_sub = np.maximum(data_sub, 0)  # Clip negative values
    
    # Clip to percentile range
    vmin, vmax = np.percentile(data_sub, percentile)
    clipped = np.clip(data_sub, vmin, vmax)
    
    # Normalize to 0-1 range
    if vmax > vmin:
        normalized = (clipped - vmin) / (vmax - vmin)
    else:
        normalized = clipped
    
    print(f"  Background subtracted: {background:.6f}")
    print(f"  Clipped to percentiles {percentile}: [{vmin:.6f}, {vmax:.6f}]")
    print(f"  After normalization: [{np.min(normalized):.6f}, {np.max(normalized):.6f}]")
    
    return normalized, vmin, vmax


def logarithmic_stretch(data, factor=1000):
    """Aggressive logarithmic stretch for faint details"""
    data_safe = np.maximum(data, 1e-10)
    stretched = np.log(1 + factor * data_safe) / np.log(1 + factor)
    return stretched


def hyperbolic_stretch(data, beta=0.05):
    """Hyperbolic stretch optimized for faint structures"""
    stretched = data / np.sqrt(data**2 + beta**2)
    # Normalize to 0-1
    stretched = (stretched - np.min(stretched)) / (np.max(stretched) - np.min(stretched))
    return stretched


def asinh_stretch(data, softening=0.1):
    """Aggressive asinh stretch for wide dynamic range"""
    stretched = np.arcsinh(data / softening) / np.arcsinh(1.0 / softening)
    stretched = np.clip(stretched, 0, 1)
    return stretched


def gamma_stretch(data, gamma=2.2):
    """Gamma correction - simple power law"""
    stretched = np.power(data, 1.0/gamma)
    return stretched


def adaptive_asinh_stretch(data, q=0.002, stretch=3.0):
    """
    FIXED: Adaptive asinh stretch (similar to LUPTON algorithm)
    Now properly calibrated to show faint structures
    q = 0.002 (was 0.01) - more aggressive
    stretch = 3.0 (was 0.5) - much brighter output
    """
    data_safe = np.maximum(data, 0)
    
    # Apply asinh stretch
    stretched = np.arcsinh(data_safe / q) / np.arcsinh(1.0 / q)
    
    # Apply additional linear stretch
    stretched = stretched * stretch
    
    # Normalize to 0-1
    stretched = np.clip(stretched, 0, 1)
    
    return stretched


def apply_all_stretches(data, params):
    """Apply all stretch methods (midtone removed)"""
    stretches = {}
    
    print("  Applying logarithmic stretch...")
    stretches['log'] = logarithmic_stretch(data, factor=params['log']['factor'])
    
    print("  Applying hyperbolic stretch...")
    stretches['hyperbolic'] = hyperbolic_stretch(data, beta=params['hyperbolic']['beta'])
    
    print("  Applying asinh stretch...")
    stretches['asinh'] = asinh_stretch(data, softening=params['asinh']['softening'])
    
    print("  Applying gamma stretch...")
    stretches['gamma'] = gamma_stretch(data, gamma=params['gamma']['gamma'])
    
    print("  Applying adaptive asinh stretch (FIXED)...")
    stretches['adaptive_asinh'] = adaptive_asinh_stretch(
        data, 
        q=params['adaptive_asinh']['q'],
        stretch=params['adaptive_asinh']['stretch']
    )
    
    return stretches


def save_stretched_images(stretches, filter_name, header, output_path):
    """Save stretched images as FITS files"""
    Path(output_path).mkdir(parents=True, exist_ok=True)
    
    for method, stretched_data in stretches.items():
        filename = f"stretched_{method}_{OBJECT_NAME}_{filter_name}.fits"
        filepath = Path(output_path) / filename
        
        # Update header
        new_header = header.copy()
        new_header['STRETCH'] = method.upper()
        new_header['HISTORY'] = f'Applied {method} stretch'
        
        # Save
        fits.writeto(filepath, stretched_data.astype(np.float32), new_header, overwrite=True)
        print(f"  Saved: {filename}")


def create_comparison_plot(original, stretches, filter_name, output_path):
    """Create comparison visualization"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    fig.suptitle(f'{OBJECT_NAME} - {filter_name.upper()} Filter Stretch Comparison', 
                 fontsize=18, fontweight='bold')
    
    # Original (normalized)
    im0 = axes[0, 0].imshow(original, cmap='gray', origin='lower', vmin=0, vmax=1)
    axes[0, 0].set_title('Original (Normalized)', fontsize=12, fontweight='bold')
    axes[0, 0].axis('off')
    plt.colorbar(im0, ax=axes[0, 0], fraction=0.046)
    
    # Plot all stretches
    methods = ['log', 'hyperbolic', 'asinh', 'gamma', 'adaptive_asinh']
    positions = [(0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
    
    for method, (row, col) in zip(methods, positions):
        im = axes[row, col].imshow(stretches[method], cmap='gray', origin='lower', vmin=0, vmax=1)
        title = f'{method.replace("_", " ").title()}'
        if method == 'adaptive_asinh':
            title += ' (FIXED)'
        axes[row, col].set_title(title, fontsize=12, fontweight='bold')
        axes[row, col].axis('off')
        plt.colorbar(im, ax=axes[row, col], fraction=0.046)
    
    plt.tight_layout()
    
    # Save comparison
    comparison_file = Path(output_path) / f"comparison_{OBJECT_NAME}_{filter_name}.png"
    plt.savefig(comparison_file, dpi=150, bbox_inches='tight')
    print(f"  Saved comparison: {comparison_file.name}")
    plt.close()


def create_histogram_plot(original, stretches, filter_name, output_path):
    """Create histogram comparison"""
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    fig.suptitle(f'{OBJECT_NAME} - {filter_name.upper()} Histograms', 
                 fontsize=18, fontweight='bold')
    
    # Original histogram
    axes[0, 0].hist(original.flatten(), bins=256, range=(0, 1), 
                    color='blue', alpha=0.7, log=True)
    axes[0, 0].set_title('Original', fontweight='bold')
    axes[0, 0].set_xlabel('Pixel Value')
    axes[0, 0].set_ylabel('Count (log scale)')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot all stretch histograms
    methods = ['log', 'hyperbolic', 'asinh', 'gamma', 'adaptive_asinh']
    positions = [(0, 1), (0, 2), (1, 0), (1, 1), (1, 2)]
    colors = ['red', 'green', 'orange', 'purple', 'cyan']
    
    for method, (row, col), color in zip(methods, positions, colors):
        axes[row, col].hist(stretches[method].flatten(), bins=256, range=(0, 1),
                           color=color, alpha=0.7, log=True)
        title = method.replace("_", " ").title()
        if method == 'adaptive_asinh':
            title += ' (FIXED)'
        axes[row, col].set_title(title, fontweight='bold')
        axes[row, col].set_xlabel('Pixel Value')
        axes[row, col].set_ylabel('Count (log scale)')
        axes[row, col].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save histogram
    hist_file = Path(output_path) / f"histogram_{OBJECT_NAME}_{filter_name}.png"
    plt.savefig(hist_file, dpi=150, bbox_inches='tight')
    print(f"  Saved histogram: {hist_file.name}")
    plt.close()


def process_filter(filter_name):
    """Process one filter through all stretches"""
    print(f"\n{'='*60}")
    print(f"Processing {filter_name.upper()} filter")
    print(f"{'='*60}")
    
    # Load image
    data, header = load_normalized_image(filter_name)
    
    # Normalize with background subtraction
    normalized_data, vmin, vmax = normalize_data(data, percentile=CLIP_PERCENTILE)
    
    # Apply stretches
    stretches = apply_all_stretches(normalized_data, STRETCH_PARAMS)
    
    # Save stretched images
    print("\nSaving stretched FITS files...")
    save_stretched_images(stretches, filter_name, header, OUTPUT_PATH)
    
    # Create comparison plot
    print("\nCreating comparison plot...")
    create_comparison_plot(normalized_data, stretches, filter_name, OUTPUT_PATH)
    
    # Create histogram plot
    print("\nCreating histogram plot...")
    create_histogram_plot(normalized_data, stretches, filter_name, OUTPUT_PATH)
    
    print(f"\n✓ Completed {filter_name} filter")
    
    return stretches


def main():
    """Main processing function"""
    print("="*60)
    print("FIXED GALAXY IMAGE STRETCHING - NGC1365")
    print("="*60)
    print(f"\nInput path: {BASE_PATH}")
    print(f"Output path: {OUTPUT_PATH}")
    print(f"\nStretch parameters:")
    print(f"  Logarithmic factor: {STRETCH_PARAMS['log']['factor']}")
    print(f"  Hyperbolic beta: {STRETCH_PARAMS['hyperbolic']['beta']}")
    print(f"  Asinh softening: {STRETCH_PARAMS['asinh']['softening']}")
    print(f"  Gamma: {STRETCH_PARAMS['gamma']['gamma']}")
    print(f"  Adaptive asinh q: {STRETCH_PARAMS['adaptive_asinh']['q']} (FIXED: was 0.01)")
    print(f"  Adaptive asinh stretch: {STRETCH_PARAMS['adaptive_asinh']['stretch']} (FIXED: was 0.5)")
    print(f"\nPercentile clipping: {CLIP_PERCENTILE}")
    print("\n⚠ Midtone stretch removed (was causing overexposure)")
    
    # Process all filters
    all_stretches = {}
    for filter_name in FILTERS:
        try:
            stretches = process_filter(filter_name)
            all_stretches[filter_name] = stretches
        except Exception as e:
            print(f"\n✗ Error processing {filter_name}: {str(e)}")
            import traceback
            traceback.print_exc()
            continue
    
    print("\n" + "="*60)
    print("PROCESSING COMPLETE!")
    print("="*60)
    print(f"\nOutput files saved to: {OUTPUT_PATH}")
    print("\nKey changes in this version:")
    print("  ✓ Adaptive Asinh FIXED - now properly bright")
    print("  ✓ Midtone removed - was causing washout")
    print("  ✓ 5 stretch methods: log, hyperbolic, asinh, gamma, adaptive_asinh")
    print("\nRecommendations:")
    print("  • Check 'adaptive_asinh' - should now show faint features clearly")
    print("  • 'gamma' or 'asinh' are good alternatives")
    print("  • Review comparison images to choose best method")
    print("\nNext steps:")
    print("1. Review comparison PNG files")
    print("2. If adaptive_asinh still needs adjustment:")
    print("   - Decrease 'q' to 0.001 for more detail")
    print("   - Increase 'stretch' to 5.0 for brighter output")
    print("3. Once happy, proceed with alignment")


if __name__ == "__main__": 
    main()

FIXED GALAXY IMAGE STRETCHING - NGC1365

Input path: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/normalized
Output path: /home/devika/PhD/S2/Obs_Astronomy/Image_Processing/Ckoirama_2025-12-19/Image_reduction_workspace/STACKED/stretched

Stretch parameters:
  Logarithmic factor: 100
  Hyperbolic beta: 0.5
  Asinh softening: 0.1
  Gamma: 2.2
  Adaptive asinh q: 0.01 (FIXED: was 0.01)
  Adaptive asinh stretch: 1 (FIXED: was 0.5)

Percentile clipping: (0.1, 99.9)

⚠ Midtone stretch removed (was causing overexposure)

Processing CLEAR filter
Loaded clear: shape=(2048, 2048), dtype=float64
  Range: [0.000000, 1.000000]
  Median: 0.019600, Std: 0.004628
  Background subtracted: 0.018566
  Clipped to percentiles (0.1, 99.9): [0.000000, 0.014818]
  After normalization: [0.000000, 1.000000]
  Applying logarithmic stretch...
  Applying hyperbolic stretch...
  Applying asinh stretch...
  Applying gamma stretch...
  Applying adaptive asin